In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#IS4

In [3]:
#Read the data

GenomicFeatures_IS4 = pd.read_csv(open('data/IS4/GenomicFeatures_IS4.txt','r'),sep='\t')
SNVCalls_IS4    = pd.read_csv(open('data/IS4/SNVCalls_IS4.txt','r'),sep='\t') 

/usr/lib/python2.7/dist-packages/pandas/io/parsers.py:1070: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
print GenomicFeatures_IS4.shape
print SNVCalls_IS4.shape

(129104, 16)
(129104, 226)


In [5]:
np.sum(GenomicFeatures_IS4.isnull())

CHROM                  0
POS                    0
END                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition          13
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    10612
Distance               0
StrandBias            13
dtype: int64

In [6]:
GenomicFeatures_IS4['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_IS4['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_IS3.isnull())

In [7]:
np.unique(GenomicFeatures_IS4['GenomicLocation'])

array(['UTR3', 'UTR5', 'UTR5;UTR3', 'downstream', 'exonic',
       'exonic;splicing', 'intergenic', 'intronic', 'nan', 'ncRNA_UTR3',
       'ncRNA_UTR5', 'ncRNA_exonic', 'ncRNA_intronic', 'ncRNA_splicing',
       'splicing', 'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [8]:
GenomicFeatures_IS4.head()

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,10392,10393,12,4,452,22,19,0.0,34.5,CCT,5.204082,0.579208,intergenic,41086,0.636364
1,1,10399,10400,11,4,473,21,19,34.0,40.0,CCT,5.111111,0.594059,NaN,41079,0.666667
2,1,20466,20467,3,3,193,6,3,23.5,63.0,GCT,2.739130,0.475248,ncRNA_intronic,31012,0.000000
3,1,61498,61499,17,3,636,20,10,47.5,39.0,ACC,2.891304,0.257426,intergenic,4726,0.450000
4,1,61850,61851,12,6,511,18,11,47.5,63.0,ATA,3.507463,0.405941,intergenic,4374,0.555556


In [9]:
SNVCalls_IS4.head()

,CHROM,POS,END,X2502792,X2502793,X2505745,X2506859,X2506861,X2507098,X2507890,X2508830,X2510262,X2510272,X2510297,X2510567,X2510929,X2511086,X2511088,X2511090,X2511092,
0,1,10392,10393,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...
1,1,10399,10400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,1,20466,20467,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,1,61498,61499,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,1,61850,61851,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,...


In [10]:
np.unique(GenomicFeatures_IS4['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [11]:
IS4 = pd.merge(GenomicFeatures_IS4, SNVCalls_IS4,left_on=['CHROM', 'POS'], right_on=['CHROM','POS'])


In [12]:
IS4.head() #182+15

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,END_y,X2502792,X2502793,X2505745,
0,1,10392,10393,12,4,452,22,19,0.0,34.5,CCT,5.204082,0.579208,intergenic,41086,0.636364,10393,0,0,0,...
1,1,10399,10400,11,4,473,21,19,34.0,40.0,CCT,5.111111,0.594059,NaN,41079,0.666667,10400,0,0,0,...
2,1,20466,20467,3,3,193,6,3,23.5,63.0,GCT,2.739130,0.475248,ncRNA_intronic,31012,0.000000,20467,0,0,0,...
3,1,61498,61499,17,3,636,20,10,47.5,39.0,ACC,2.891304,0.257426,intergenic,4726,0.450000,61499,0,0,0,...
4,1,61850,61851,12,6,511,18,11,47.5,63.0,ATA,3.507463,0.405941,intergenic,4374,0.555556,61851,0,0,0,...


In [13]:
IS4_X = IS4.iloc[:,3:16] #Exclude CHROM, POS
IS4_Y = IS4.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [14]:
print IS4_X.shape
print IS4_Y.shape

(112720, 13)
(112720, 223)


In [15]:
IS4_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,12,4,452,22,19,0,34.5,CCT,5.204082,0.579208,intergenic,41086,0.636364
1,11,4,473,21,19,34,40.0,CCT,5.111111,0.594059,NaN,41079,0.666667


In [16]:
IS4_Y[:2]

,X2502792,X2502793,X2505745,X2506859,X2506861,X2507098,X2507890,X2508830,X2510262,X2510272,X2510297,X2510567,X2510929,X2511086,X2511088,X2511090,X2511092,X2511094,X2511096,X2511098,
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [17]:
totalSample = IS4_Y.shape[0]
IS4_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print IS4_Y_Val.shape

112720
(112720, 1)


In [18]:
IS4_Y.shape

(112720, 223)

In [19]:
IS4_Y.values[0].shape

(223,)

In [20]:
for i in range(totalSample):
    IS4_Y_Val[i] = np.argmax(np.bincount(IS4_Y.values[i]))

In [21]:
IS4_Y_Val_1 = pd.DataFrame(IS4_Y_Val)
IS4_Y_Val_1.columns = ['Label']

In [22]:
IS4_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [23]:
IS4_dataframe = pd.concat([IS4_X, IS4_Y_Val_1],axis=1)

In [24]:
IS4_dataframe.shape

(112720, 14)

In [25]:
IS4_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
0,12,4,452,22,19,0,34.5,CCT,5.204082,0.579208,intergenic,41086,0.636364,0
1,11,4,473,21,19,34,40.0,CCT,5.111111,0.594059,NaN,41079,0.666667,0


In [26]:
IS4_dataframe.to_csv('IS4.csv')

In [29]:
#Now we have variable importance ...lets do analysis

In [28]:
%load_ext rpy2.ipython

In [29]:
%%R
# Read in the data
IS4Frame = read.csv("IS4.csv")
str(IS4Frame)

'data.frame':	112720 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  12 11 3 17 12 30 49 51 15 14 ...
 $ NonRefAllele   : int  4 4 3 3 6 13 12 10 8 4 ...
 $ BaseQual       : int  452 473 193 636 511 1190 954 1142 709 562 ...
 $ TumourCoverage : int  22 21 6 20 18 45 67 66 23 18 ...
 $ NormalCoverage : int  19 19 3 10 11 40 66 66 16 19 ...
 $ MapQual        : num  0 34 23.5 47.5 47.5 60 60 60 40 43 ...
 $ ReadPosition   : num  34.5 40 63 39 63 21 38 40 50 51.5 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 12 12 20 2 5 32 29 5 26 32 ...
 $ HomopolymerRate: num  5.2 5.11 2.74 2.89 3.51 ...
 $ GCcontent      : num  0.579 0.594 0.475 0.257 0.406 ...
 $ GenomicLocation: Factor w/ 17 levels "","downstream",..: 5 1 8 5 5 5 5 5 5 5 ...
 $ Distance       : int  41086 41079 31012 4726 4374 109 145 147 54 7196 ...
 $ StrandBias     : num  0.636 0.667 0 0.45 0.556 ...
 $ Label          : num  0 0 0 0 0 0 0 0 0 0 ...


In [30]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(IS4Frame)) 

[1] 0


In [33]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [31]:
%%R
sum(is.na(IS4Frame))  #No NA value now

[1] 0


In [32]:
%%R
write.csv(IS4Frame, file = "IS4_imputed.csv")  #Cleaned CSV without NA

In [33]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         34 minutes 39 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.59 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [34]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/IS4_imputed.csv"
IS4.hex          = h2o.importFile(path = csvPath, destination_frame = "IS4.hex") 
train                 = as.data.frame(IS4.hex)

str(train)

  |======================================================================| 100%
'data.frame':	112720 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  12 11 3 17 12 30 49 51 15 14 ...
 $ NonRefAllele   : int  4 4 3 3 6 13 12 10 8 4 ...
 $ BaseQual       : int  452 473 193 636 511 1190 954 1142 709 562 ...
 $ TumourCoverage : int  22 21 6 20 18 45 67 66 23 18 ...
 $ NormalCoverage : int  19 19 3 10 11 40 66 66 16 19 ...
 $ MapQual        : num  0 34 23.5 47.5 47.5 60 60 60 40 43 ...
 $ ReadPosition   : num  34.5 40 63 39 63 21 38 40 50 51.5 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 12 12 20 2 5 32 29 5 26 32 ...
 $ HomopolymerRate: num  5.2 5.11 2.74 2.89 3.51 ...
 $ GCcontent      : num  0.579 0.594 0.475 0.257 0.406 ...
 $ GenomicLocation: Factor w/ 16 levels "downstream","exonic",..: 4 NA 7 4 4 4 4 4 4 4 ...
 $ Distance       : int  41086 41079 31012 4726 4374 109 

In [35]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [36]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [37]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.360420
2       BaseQual            0.238011          0.238011   0.085784
3      RefAllele            0.185920          0.185920   0.067009
4 NormalCoverage            0.154091          0.154091   0.055537
5 TumourCoverage            0.152289          0.152289   0.054888

---
                     variable relative_importance scaled_importance percentage
56          Trinucleotide.TCC            0.005665          0.005665   0.002042
57 GenomicLocation.intergenic            0.005408          0.005408   0.001949
58          Trinucleotide.GCT            0.005073          0.005073   0.001828
59          Trinucleotide.CCA            0.004997          0.004997   0.001801
60                 StrandBias            0.004259          0.004259   0.001535
61  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [38]:
%%R
write.table(var_importance, file="IS4_variable_importance.csv")

In [39]:
var_importance = pd.read_csv(open('IS4_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [40]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.360420
2,BaseQual,0.238011,0.238011,0.085784
3,RefAllele,0.185920,0.185920,0.067009
4,NormalCoverage,0.154091,0.154091,0.055537
5,TumourCoverage,0.152289,0.152289,0.054888


In [41]:
GenomicFeatures_IS4[:5]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,10392,10393,12,4,452,22,19,0.0,34.5,CCT,5.204082,0.579208,intergenic,41086,0.636364
1,1,10399,10400,11,4,473,21,19,34.0,40.0,CCT,5.111111,0.594059,NaN,41079,0.666667
2,1,20466,20467,3,3,193,6,3,23.5,63.0,GCT,2.739130,0.475248,ncRNA_intronic,31012,0.000000
3,1,61498,61499,17,3,636,20,10,47.5,39.0,ACC,2.891304,0.257426,intergenic,4726,0.450000
4,1,61850,61851,12,6,511,18,11,47.5,63.0,ATA,3.507463,0.405941,intergenic,4374,0.555556


In [42]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.23801118135452301,
 'Distance': 0.0079919714480638504,
 'GCcontent': 0.0194189436733723,
 'GenomicLocation.UTR3': 0.0139772212132812,
 'GenomicLocation.UTR5': 0.019093381240963901,
 'GenomicLocation.UTR5;UTR3': 0.041426029056310702,
 'GenomicLocation.downstream': 0.0124378781765699,
 'GenomicLocation.exonic': 0.024452332407236099,
 'GenomicLocation.exonic;splicing': 0.0413472168147564,
 'GenomicLocation.intergenic': 0.0054084528237581296,
 'GenomicLocation.intronic': 0.0062452950514853001,
 'GenomicLocation.missing(NA)': 0.0652158558368683,
 'GenomicLocation.ncRNA_UTR3': 0.037007015198469197,
 'GenomicLocation.ncRNA_UTR5': 0.038546890020370497,
 'GenomicLocation.ncRNA_exonic': 0.022588627412915199,
 'GenomicLocation.ncRNA_intronic': 0.0125372279435396,
 'GenomicLocation.ncRNA_splicing': 0.033338088542222998,
 'GenomicLocation.splicing': 0.037821322679519702,
 'GenomicLocation.upstream': 0.015775028616190002,
 'GenomicLocation.upstream;downstream': 0.0336332954466343,
 'H

In [43]:
GenomicFeatures_IS4.columns

Index([u'CHROM', u'POS', u'END', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'Distance', u'StrandBias'], dtype='object')

In [44]:
GenomicFeatures_IS4.dtypes

CHROM               object
POS                  int64
END                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
Distance             int64
StrandBias         float64
dtype: object

In [45]:
GenomicFeatures_IS4.iloc[0,13]

'intergenic'

In [46]:
GenomicFeatures_IS4[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,10392,10393,12,4,452,22,19,0,34.5,CCT,5.204082,0.579208,intergenic,41086,0.636364
1,1,10399,10400,11,4,473,21,19,34,40.0,CCT,5.111111,0.594059,NaN,41079,0.666667


In [48]:
noOfSample = GenomicFeatures_IS4.shape[0]

IS4_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_IS4.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_IS4.iloc[i,3]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_IS4.iloc[i,4]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_IS4.iloc[i,5]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_IS4.iloc[i,6]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_IS4.iloc[i,7]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_IS4.iloc[i,8]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_IS4.iloc[i,9]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_IS4.iloc[i,10], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_IS4.iloc[i,10])]
    HomopolymerRate= GenomicFeatures_IS4.iloc[i,11]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_IS4.iloc[i,12]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_IS4.iloc[i,13], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_IS4.iloc[i,13])]
    Distance    = GenomicFeatures_IS4.iloc[i,14]*var_imp_dict['Distance']
    StrandBias     = GenomicFeatures_IS4.iloc[i,15]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    IS4_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+Distance+StrandBias

In [49]:
#Sorting dictionary
IS4_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in IS4_Dict.items()],reverse=True)]

In [50]:
IS4_Dict_Frame = pd.DataFrame(IS4_Dict1, columns=['POS', 'Value'])

In [51]:
IS4_Dict_Frame.to_csv("IS4_Dict_Frame.csv")

In [52]:
IS4_Dict_Frame[:10]

,POS,Value
0,121485153,81797.940172
1,121485161,81611.688349
2,121485255,80631.109386
3,121485309,80291.004489
4,121485254,79853.609672
5,121485114,78497.920492
6,121485111,78254.580406
7,121485196,78235.346911
8,121484958,78169.229084
9,121485214,78156.653039


In [56]:
GenomicFeatures_IS4_1 = pd.concat([IS4.iloc[:,0:16], IS4_Y_Val_1],axis=1)

In [57]:
rows = IS4_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_IS4_Top20 = GenomicFeatures_IS4_1.loc[GenomicFeatures_IS4_1['POS'].isin(rows)]

[121485153 121485161 121485255 121485309 121485254 121485114 121485111
 121485196 121484958 121485214 121484934 121484642 121485314 121485078
 121484654 121485076 121484491 121484959  61969356 121484813]


In [59]:
GenomicFeatures_IS4_Top20[:10]

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
5196,1,121484491,121484492,7965,32,275461,30768,31331,60,48,CTG,2.544828,0.361386,NaN,1556,0.504648,0
5205,1,121484642,121484643,7884,48,271591,37492,38086,57,53,GTA,2.655172,0.376238,NaN,1707,0.543023,0
5206,1,121484654,121484655,7902,35,271391,35279,35614,55,54,ACC,2.486486,0.371287,NaN,1719,0.539925,0
5214,1,121484813,121484814,7946,32,272823,32695,33074,60,51,GTC,2.692857,0.386139,NaN,1878,0.501269,0
5223,1,121484934,121484935,7905,37,278611,32321,32664,60,49,CTG,2.434483,0.371287,NaN,1999,0.517868,0
5225,1,121484958,121484959,7913,38,280335,32254,32485,60,52,CTC,2.426573,0.391089,NaN,2023,0.510572,0
5226,1,121484959,121484960,7896,38,273582,32281,32490,60,52,TCA,2.482517,0.396040,NaN,2024,0.511291,0
5231,1,121485076,121485077,7845,38,270614,35241,35574,57,40,CTG,2.116883,0.400990,NaN,2141,0.475157,0
5232,1,121485078,121485079,7801,70,271795,36018,36315,51,38,GCT,2.124183,0.400990,NaN,2143,0.470348,0
5233,1,121485111,121485112,7937,60,267989,42148,42157,42,45,GTT,2.248322,0.405941,NaN,2176,0.429178,0


In [55]:
GenomicFeatures_IS4_Top20.to_csv('GenomicFeatures_IS4_Top20.csv')